In [ ]:
import random
from mido import Message, MidiFile, MidiTrack
import stilus.midi.utils as utl

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from stilus.data.sets import MidiDataset
from torch.utils.data import DataLoader

In [ ]:
#import a midi file
mid = MidiFile("midi/training/mary.mid")

In [ ]:
# print all events
for i, track in enumerate(mid.tracks):
    print('Track {}: {}'.format(i, track.name))
    for msg in track:
        #if msg.type == "note_on":
        print(msg)

In [ ]:
tensor = utl.convert_midi_to_time_series(mid,5,5,8)
print(tensor[:,0:9])

In [ ]:
class ConvNet(nn.Module):

    def __init__(self):
        super(ConvNet, self).__init__() # bs * 5 * 32
        self.conv1 = nn.Conv1d(5, 10, 4) # bs * 10 * 28+1
        self.fc0 = nn.Linear(10*29, 64) # bs * 64
        self.conv2 = nn.Conv1d(1, 10, 4) # bs * 10 * 60
        # maxpool would be here convert it to bs * 10 * 30
        self.fc1 = nn.Linear(10 * 30, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 5)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc0(x))
        x = torch.unsqueeze(x, 1)
        x = F.relu(self.conv2(x))
        x = F.max_pool1d(x,2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features


In [ ]:
conv_net = ConvNet()
conv_net.load_state_dict(torch.load("models/50epochs_1.0.2.pth"))
conv_net.eval()

In [ ]:
midi_dataset = MidiDataset("training_data.npy")
dataloader = DataLoader(midi_dataset, batch_size=32, shuffle=True)
print(midi_dataset.std)
print(midi_dataset.mean)

In [ ]:
def std_tensor_to_int(pred):
    return ((pred * midi_dataset.std) + midi_dataset.mean).int()

In [ ]:
def int_to_std_tensor(input):
    return ((input - midi_dataset.mean) / midi_dataset.std)

In [ ]:
print(tensor.shape)
std_tensor = int_to_std_tensor(tensor[:,0:0+32].astype("float32"))
print(std_tensor.shape)
tensor_in = torch.unsqueeze(torch.from_numpy(std_tensor),0)
print("tensor_in", tensor_in)
#print(tensor_in.dtype)
pred = conv_net(tensor_in)
print("pred:",  std_tensor_to_int(pred))

In [ ]:
#How to create a midi file from scratch
notes = [64, 64+7, 64+12]

outfile = MidiFile()

track = MidiTrack()
outfile.tracks.append(track)

track.append(Message('program_change', program=12))

delta = 32
for i in range(200):
    note = random.choice(notes)
    track.append(Message('note_on', note=note, velocity=100, time=delta))
    track.append(Message('note_off', note=note, velocity=100, time=delta))

outfile.save('midi/new_song.mid')

In [ ]:
outfile.length